In [1]:
from utz import *
from nj_crashes.paths import ROOT_DIR
from njsp import crashes
from njsp.paths import PROJECTED_CSV
from njsp.ytc import to_ytc
from njsp.ytd import oldest_commit_rundate_since, projected_roy_deaths
from njsp import Ytd

chdir(ROOT_DIR)

In [2]:
ytd = Ytd()
prv_rundate = ytd.prv_rundate
prv_rundate

Searching for oldest commit with rundate ≥2024-12-18


Found rundate 2024-12-17 10:00:00 < 2024-12-18 at commit 37ed4f57; returning commit ed2db4e


'2024-12-18 10:00:01'

In [3]:
prv_ytd_ratio = ytd.cur_ytd_deaths / ytd.prv_ytd_deaths
pct_change = (prv_ytd_ratio - 1) * 100

print(f'As of {ytd.prv_rundate}, NJSP was reporting {ytd.prv_ytd_total} YTD deaths')
print(f'Current YTD Deaths ({ytd.rundate}): {ytd.cur_ytd_deaths}')
print(f'Previous year YTD Deaths (adjusted; {ytd.prv_rundate}): {ytd.prv_ytd_deaths}')
print(f'Projected {ytd.cur_year} total: {ytd.projected_year_total}')
print(f'{pct_change:.1f}% change')

As of 2024-12-18 10:00:01, NJSP was reporting 669 YTD deaths
Current YTD Deaths (2025-12-18 10:00:55-05:00): 543
Previous year YTD Deaths (adjusted; 2024-12-18 10:00:01): 668.9305444801794
Projected 2025 total: 555.3381021407865
-18.8% change


322616f500c4a1edeb84bf9dda1b33d463008097: FAUQStats cache miss: 2024, Wed Dec 18 10:00:01 EST 2024


Find the oldest commit with rundate less than 1 year ago, load crashes from that time:

In [4]:
prv_crashes = ytd.prv_ytd_crashes
prv_crashes

,CCODE,CNAME,MCODE,MNAME,STREET,LOCATION,FATALITIES,FATAL_D,FATAL_P,FATAL_T,FATAL_B,HIGHWAY,INJURIES,dt
ACCID,,,,,,,,,,,,,,
12918,12,Middlesex,1218,Plainsboro Twsp,NaN,State Highway 1,1.0,0.0,0.0,1.0,0.0,1,4.0,2024-01-01 01:56:00-05:00
12923,11,Mercer,1111,Trenton,NaN,State Highway 129,1.0,1.0,0.0,0.0,0.0,129,NaN,2024-01-03 12:14:00-05:00
12932,01,Atlantic,0112,Hamilton Twsp,NaN,Atlantic City Expressway MP 20.5,1.0,1.0,0.0,0.0,0.0,446,0.0,2024-01-05 09:14:00-05:00
12933,02,Bergen,0248,Ramsey Boro,East Oak St,East Oak St,1.0,1.0,0.0,0.0,0.0,NaN,0.0,2024-01-05 14:12:00-05:00
13362,07,Essex,0701,Belleville Town,NaN,County 647 MP .76,1.0,0.0,0.0,1.0,0.0,647,NaN,2024-01-05 18:08:00-05:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13650,01,Atlantic,0102,Atlantic City,NaN,Atlantic City Expressway MP 1.4,1.0,1.0,0.0,0.0,0.0,446,NaN,2024-12-12 07:55:00-05:00
13651,19,Sussex,1914,Montague Twsp,NaN,County 653 MP 2.1,1.0,1.0,0.0,0.0,0.0,653,0.0,2024-12-12 13:22:00-05:00
13652,03,Burlington,0330,Pemberton Twsp,Ridge Rd,Ridge Rd,1.0,1.0,0.0,0.0,0.0,NaN,NaN,2024-12-14 11:31:00-05:00


Load previous year YTD and final counts, and current year YTD counts:

In [5]:
prv_year = ytd.prv_year
cur_year = ytd.cur_year

prv_ytc = to_ytc(prv_crashes)
prv_ytd = prv_ytc.loc[prv_year]

prv_end = to_ytc(ytd.prv_end_crashes).loc[prv_year]

cur_ytc = to_ytc(ytd.cur_ytd_crashes)
cur_ytd = cur_ytc.loc[cur_year]

cur_ytd

,driver,passenger,pedestrian,cyclist,crashes
county,,,,,
Atlantic,17,4,12,0,32
Bergen,22,2,5,0,29
Burlington,23,5,8,1,35
Camden,20,3,15,2,39
Cape May,10,4,0,0,12
Cumberland,10,2,8,0,19
Essex,10,5,24,1,40
Gloucester,18,6,3,1,25
Hudson,7,1,7,2,15


Compute fraction of current year that has elapsed (year-to-date; "YTD") and still remains (rest-of-year; "ROY"):

In [6]:
cur_ytd_frac = ytd.cur_year_frac
cur_roy_frac = 1 - cur_ytd_frac
print('%.1f%% through the year, %.1f%% remaining' % (ytd.cur_year_frac * 100, ytd.cur_roy_frac * 100))

96.3% through the year, 3.7% remaining


Combine previous year YTD / end with current year YTD:

In [7]:
def melt(df, name):
    return (
        df
        .melt(ignore_index=False, var_name='type')
        .set_index('type', append=True)
        .value
        .rename(name)
    )

z = sxs(
    melt(prv_ytd, 'prv_ytd'),
    melt(prv_end, 'prv_end'),
    melt(cur_ytd, 'cur_ytd'),
).fillna(0).astype(int)
z

,,prv_ytd,prv_end,cur_ytd
county,type,,,
Atlantic,driver,30,32,17
Bergen,driver,15,14,22
Burlington,driver,31,31,23
Camden,driver,18,19,20
Cape May,driver,8,8,10
...,...,...,...,...
Salem,crashes,13,14,18
Somerset,crashes,19,18,12
Sussex,crashes,13,12,11


In [8]:
def project_roy(r):
    return int(round(projected_roy_deaths(r.prv_ytd, r.prv_end, r.cur_ytd, cur_ytd_frac)))

roy = z.apply(project_roy, axis=1).rename('roy')
projected = (
    (z.cur_ytd + roy)
    .rename('projected')
    .reset_index(level=1)
    .pivot(columns='type', values='projected')
)
projected

type,crashes,cyclist,driver,passenger,pedestrian
county,,,,,
Atlantic,33,0,18,4,13
Bergen,30,0,21,2,6
Burlington,36,1,23,5,9
Camden,40,2,21,4,14
Cape May,13,0,10,4,0
Cumberland,18,0,9,2,8
Essex,41,1,11,5,23
Gloucester,26,1,19,6,3
Hudson,17,0,7,1,11


In [9]:
projected.to_csv(PROJECTED_CSV)